# Loads the dataset

In [1]:
import pandas as pd
import numpy as np

output_2Y = pd.read_csv('1-output_2Y.csv')
print(output_2Y.shape)
output_2Y.head(3)

(1056977, 26)


,AUTOR,CANCELADA,RECLAMACION,NODO,RESOLUTOR,DESCORG,FECHAMOD,TIPOLOGIA,ASUNTO,ASIGNATARIO,...,SOPORTE,SISTEMA,GRUPOAUTOR,ESTADO,CENTRO,ORIGEN,REAPERTURAINC,TYPEORG,RECURSO,MOTIVOCIERRE
0,M; P A,0,0,CORDOBA,CEGES OPERACIONES,Create a new request/incident/problem/change/i...,2014-10-01T02:50:38+0200,Incidencia.Sistema,Datos de i,MORENO; PARRILLA A,...,0,CA-SDM,CEGES OPERACIONES,Abierta,H. U. REINA SOFIA,TELEFONO,0,INIT,GIPI,NaN
1,S; R E,0,0,CORDOBA,CEGES OPERACIONES,Create a new request/incident/problem/change/i...,2014-10-01T02:50:45+0200,Incidencia.Sistema,Problemas,SANCHEZ; ROLDAN E,...,0,CA-SDM,CEGES OPERACIONES,Abierta,H. U. REINA SOFIA,TELEFONO,0,INIT,IMPAX_VISOR,
2,C; L C,0,0,MALAGA,CEGES OPERACIONES,Create a new request/incident/problem/change/i...,2014-10-01T02:50:51+0200,Peticion.Sistema,Instalar U,CABEZUDO; LORENZO C,...,0,CA-SDM,CEGES OPERACIONES,Abierta,H. G. BASICO ANTEQUERA,TELEFONO,0,INIT,UNIDOSIS,


In [2]:
len(output_2Y['ID'].unique())

257278

In [3]:
output = output_2Y.sort_values(['ID', 'SECUENCIA']).reset_index(drop=True)

In [4]:
resol_mask = output['RESOLUTOR'] == 'RESOLUTOR-PUESTO-USUARIO'

In [5]:
output_spu = output.loc[pd.concat([resol_mask, output['ID']], axis=1).groupby('ID').transform('sum')['RESOLUTOR'] > 0].copy()

In [6]:
output_spu.shape

(376670, 26)

In [7]:
len(output_spu['ID'].unique())

65559

In [8]:
output_spu.dtypes

AUTOR             object
CANCELADA          int64
RECLAMACION        int64
NODO              object
RESOLUTOR         object
DESCORG           object
FECHAMOD          object
TIPOLOGIA         object
ASUNTO            object
ASIGNATARIO       object
ID                 int64
SECUENCIA          int64
FECHA             object
AREA             float64
PADRE            float64
PRIORIDAD         object
SOPORTE            int64
SISTEMA           object
GRUPOAUTOR        object
ESTADO            object
CENTRO            object
ORIGEN            object
REAPERTURAINC      int64
TYPEORG           object
RECURSO           object
MOTIVOCIERRE      object
dtype: object

In [9]:
np.any(output_spu['PADRE'].unique() == 0)

False

# Preprocess and encode

In [10]:
output_spu.loc[:,'PADRE'] = output_spu.loc[:,'PADRE'].fillna(0)

In [11]:
import EventLog as el
spulog = el.EventLog(output_spu, 'ID', 'FECHA')

Found False


In [12]:
spulog.static_columns()

['CANCELADA', 'SOPORTE', 'SISTEMA', 'CENTRO', 'ORIGEN', 'REAPERTURAINC']

In [13]:
spulog.df.describe()

,CANCELADA,RECLAMACION,ID,SECUENCIA,AREA,PADRE,SOPORTE,REAPERTURAINC
count,376670.0,376670.000000,3.766700e+05,3.766700e+05,0.0,3.766700e+05,376670.000000,376670.0
mean,0.0,0.016508,2.742874e+06,1.770789e+07,NaN,2.021258e+04,0.092752,0.0
std,0.0,0.127418,6.925320e+04,4.351400e+05,NaN,2.335229e+05,0.290085,0.0
min,0.0,0.000000,1.531052e+06,1.676933e+07,NaN,0.000000e+00,0.000000,0.0
25%,0.0,0.000000,2.685230e+06,1.732713e+07,NaN,0.000000e+00,0.000000,0.0
50%,0.0,0.000000,2.745169e+06,1.772202e+07,NaN,0.000000e+00,0.000000,0.0
75%,0.0,0.000000,2.802236e+06,1.809152e+07,NaN,0.000000e+00,0.000000,0.0
max,0.0,1.000000,2.856466e+06,2.257517e+07,NaN,2.911191e+06,1.000000,0.0


In [14]:
[np.unique(spulog.df[x]) for x in ['CANCELADA', 'RECLAMACION', 'SOPORTE', 'REAPERTURAINC']]

[array([0], dtype=int64),
 array([0, 1], dtype=int64),
 array([0, 1], dtype=int64),
 array([0], dtype=int64)]

In [15]:
spulog.df.select_dtypes('object').describe()

,AUTOR,NODO,RESOLUTOR,DESCORG,FECHAMOD,TIPOLOGIA,ASUNTO,ASIGNATARIO,PRIORIDAD,SISTEMA,GRUPOAUTOR,ESTADO,CENTRO,ORIGEN,TYPEORG,RECURSO,MOTIVOCIERRE
count,376670,376670,376670,376670,376670,376670,376653,376670,376670,376670,349044,376670,376670,376670,376670,376670,104007
unique,2863,9,122,12772,333450,66,9067,724,3,1,91,6,1665,5,9,12806,245
top,P; U S,SEVILLA,RESOLUTOR-PUESTO-USUARIO,Create a new request/incident/problem/change/i...,2014-11-06T10:44:48+0100,Incidencia.Hardware,Problemas,PUESTO; USUARIO S,P3,CA-SDM,CEGES OPERACIONES,Abierta,H. U. VIRGEN DEL ROCIO,TELEFONO,TR,NO_RECEPCIONADO_HW,
freq,90801,75987,241733,66757,25,104931,41225,241715,376224,376670,121993,226068,14469,230555,137908,62047,74751


In [16]:
spulog.df['GRUPOAUTOR'].fillna('0', inplace=True)

In [17]:
np.sum((spulog.df['MOTIVOCIERRE'].value_counts()/len(spulog.df)*100) > 0.1)

23

In [18]:
np.sum(spulog.df.CENTRO.value_counts()/len(spulog.df)*100 > 0.1)

136

In [19]:
np.sum(spulog.df.CENTRO.value_counts()/len(spulog.df)*100>0.1)

136

In [20]:
from sklearn.preprocessing import OneHotEncoder

centros = spulog.df.CENTRO.value_counts()/len(spulog.df) * 100
centros_encoder = OneHotEncoder(sparse=False, categories=[centros[centros > 0.1].index.values], handle_unknown='ignore')

encoder = el.LogEncoder(transformers = [('static_drop', 'drop', ['SISTEMA']),
                                     ('static_keep', 'keep', ['SOPORTE','ID']),
                                     ('static_onehot', el.WrapperEncoder(spulog.id_column,OneHotEncoder(sparse=False)), ['ORIGEN']),
                                     ('static_onehot_filter', el.WrapperEncoder(spulog.id_column,centros_encoder), ['CENTRO']),
                                     ('dynamic_drop', 'drop', ['FECHAMOD', 'DESCORG', 'ASUNTO', 'SECUENCIA', 'PADRE', 'AREA']),
                                     ('dynamic_keep', 'keep', ['CANCELADA', 'RECLAMACION','REAPERTURAINC']),
                                     ('dynamic_freq', el.FrequencyEncoder(spulog.id_column), ['NODO', 'RESOLUTOR', 'TIPOLOGIA', 'PRIORIDAD', 'GRUPOAUTOR', 'ESTADO', 'TYPEORG']),
                                     ('timestamp', el.TimestampFeatures(spulog.id_column, ['event_order', 'time_from_start', 'elapsed_time_from_event','remaining_time']), [spulog.timestamp_column])])


encoder.check_unused(spulog)

['AUTOR', 'ASIGNATARIO', 'RECURSO', 'MOTIVOCIERRE']

In [21]:
dataset = encoder.fit_transform(spulog)
dataset.head(5)

Fitting static_drop
Fitting static_keep
Fitting static_onehot
Fitting static_onehot_filter
Fitting dynamic_drop
Fitting dynamic_keep
Fitting dynamic_freq
Fitting timestamp
Transforming static_drop
Transforming static_keep
Transforming static_onehot
Transforming static_onehot_filter
Transforming dynamic_drop
Transforming dynamic_keep
Transforming dynamic_freq
Transforming timestamp
0        2015-01-17 00:07:35+00:00
1        2015-01-17 00:07:35+00:00
2        2015-01-17 00:07:35+00:00
3        2015-01-17 00:07:35+00:00
4        2015-01-17 00:07:35+00:00
5        2015-01-17 00:07:35+00:00
6        2015-01-17 00:07:35+00:00
7        2015-01-17 00:07:35+00:00
8        2015-01-17 00:07:35+00:00
9        2015-01-17 00:07:35+00:00
10       2015-01-17 00:07:35+00:00
11       2014-10-27 15:06:24+00:00
12       2014-10-27 15:06:24+00:00
13       2014-10-27 15:06:24+00:00
14       2014-10-27 15:06:24+00:00
15       2014-10-27 15:06:24+00:00
16       2014-10-27 15:06:24+00:00
17       2014-10-27 1

,SOPORTE,ID,x0_EMAIL,x0_FAX,x0_INTRANET,x0_OTROS,x0_TELEFONO,x0_H. U. VIRGEN DEL ROCIO,x0_H. U. REINA SOFIA,x0_H. U. DE JEREZ,...,TYPEORG_INIT,TYPEORG_RE,TYPEORG_RECLAMAR,TYPEORG_RO,TYPEORG_ST,TYPEORG_TR,event_order_FECHA,time_from_start_FECHA,remaining_time_FECHA,elapsed_time_from_event_FECHA
0,0,1531052,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,1,0,0 days 00:00:00,87 days 15:17:57,0 days 00:00:00
1,0,1531052,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,2,1,0 days 00:21:57,87 days 14:56:00,0 days 00:21:57
2,0,1531052,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,3,2,0 days 00:33:27,87 days 14:44:30,0 days 00:11:30
3,0,1531052,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,4,3,6 days 02:07:51,81 days 13:10:06,6 days 01:34:24
4,0,1531052,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,5,4,6 days 02:12:45,81 days 13:05:12,0 days 00:04:54


In [22]:
np.all(dataset.isnull().sum()==0)

True

In [23]:
dataset.describe

 0  2856458       0.0     0.0          0.0       0.0   
376660        0  2856460       0.0     0.0          0.0       0.0   
376661        0  2856460       0.0     0.0          0.0       0.0   
376662        0  2856461       0.0     0.0          0.0       1.0   
376663        0  2856461       0.0     0.0          0.0       1.0   
376664        1  2856464       0.0     0.0          1.0       0.0   
376665        1  2856464       0.0     0.0          1.0       0.0   
376666        0  2856466       0.0     0.0          1.0       0.0   
376667        0  2856466       0.0     0.0          1.0       0.0   
376668        0  2856466       0.0     0.0          1.0       0.0   
376669        0  2856466       0.0     0.0          1.0       0.0   

        x0_TELEFONO  x0_H. U. VIRGEN DEL ROCIO  x0_H. U. REINA SOFIA  \
0               0.0                        0.0                   0.0   
1               0.0                        0.0                   0.0   
2               0.0                  

In [24]:
dataset.columns.values

array(['SOPORTE', 'ID', 'x0_EMAIL', 'x0_FAX', 'x0_INTRANET', 'x0_OTROS',
       'x0_TELEFONO', 'x0_H. U. VIRGEN DEL ROCIO', 'x0_H. U. REINA SOFIA',
       'x0_H. U. DE JEREZ', 'x0_H. U. VIRGEN DE LAS NIEVES',
       'x0_H. G. DE MALAGA', 'x0_H. VALLE DE LOS PEDROCHES',
       'x0_H. U. VIRGEN MACARENA', 'x0_H. U. NTRA. SRA. DE VALME',
       'x0_H. U. TORRECARDENAS',
       'x0_H. CIUDAD DE JAEN(MEDICO QUIRURGICO)',
       'x0_H. U. PUERTA DEL MAR', 'x0_H. G. BASICO BAZA',
       'x0_H. U. VIRGEN DE LA VICTORIA', 'x0_H. INFANTA MARGARITA',
       'x0_H. U. SAN CECILIO', 'x0_H. NTRA. SRA. MERCED',
       'x0_H. G. BASICO ANTEQUERA', 'x0_H. G. RIOTINTO',
       'x0_H. PUNTA EUROPA', 'x0_H. JUAN RAMON JIMENEZ',
       'x0_H. U. PUERTO REAL', 'x0_H. G. BASICO SERRANIA DE RONDA',
       'x0_H. G. B. SANTA ANA MOTRIL', 'x0_H. C. AXARQUIA',
       'x0_H. PROVINCIAL DE CORDOBA',
       'x0_H. MATERNO INFANTIL REGIONAL DE MALAGA', 'x0_H. INMACULADA',
       'x0_H. LA LINEA', 'x0_H.CIVIL', 'x0_H

# Split into train, validation, test

In [25]:
# Split into train, validation, test

from sklearn.model_selection import GroupShuffleSplit
from sklearn.utils import safe_indexing
from itertools import chain


def group_train_test_split(X, Y, group, test_size=0.2):
    train, test = next(GroupShuffleSplit(test_size=test_size, random_state=0).split(X, Y, group))
    arrays = [X, Y, group]

    return list(chain.from_iterable((safe_indexing(a, train),
                                     safe_indexing(a, test)) for a in arrays))

In [26]:
#cambiar los timedelta atipo long
        
timedeltas = dataset.select_dtypes('timedelta').columns.values
for c in timedeltas:
    dataset[c] = dataset[c].dt.seconds

incidents_X = dataset

# Choose the target attribute
incidents_Y = dataset['remaining_time_FECHA']

# And the attribute (id) that is used to split the dataset in train, validation and test
incidents_group = dataset['ID']

incidents_X.shape, incidents_Y.shape, incidents_group.shape

((376670, 457), (376670,), (376670,))

In [27]:
incidents_X

,SOPORTE,ID,x0_EMAIL,x0_FAX,x0_INTRANET,x0_OTROS,x0_TELEFONO,x0_H. U. VIRGEN DEL ROCIO,x0_H. U. REINA SOFIA,x0_H. U. DE JEREZ,...,TYPEORG_INIT,TYPEORG_RE,TYPEORG_RECLAMAR,TYPEORG_RO,TYPEORG_ST,TYPEORG_TR,event_order_FECHA,time_from_start_FECHA,remaining_time_FECHA,elapsed_time_from_event_FECHA
0,0,1531052,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,1,0,0,55077,0
1,0,1531052,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,2,1,1317,53760,1317
2,0,1531052,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,3,2,2007,53070,690
3,0,1531052,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,4,3,7671,47406,5664
4,0,1531052,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,5,4,7965,47112,294
5,0,1531052,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,6,5,7985,47092,20
6,0,1531052,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,7,6,11031,44046,3046
7,0,1531052,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,8,7,13616,41461,2585
8,0,1531052,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,9,8,82000,59477,68384
9,0,1531052,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0,1,0,0,0,0,9,85720,55757,3720


In [28]:
# Split into train, validation, test
X_train, X_test, Y_train, Y_test, group_train, group_test = group_train_test_split(incidents_X, incidents_Y, incidents_group, test_size = 0.5)
X_val, X_test, Y_val, Y_test, group_val, group_test = group_train_test_split(X_test, Y_test, group_test, test_size=0.4)

X_train.shape, Y_train.shape, X_test.shape, Y_test.shape, X_val.shape, Y_val.shape

((188259, 457), (188259,), (75365, 457), (75365,), (113046, 457), (113046,))

In [29]:
# Quick check that case ids are not in two slots
#np.asarray([item in group_test.unique() for item in group_train.unique()]).any()

# Trains classifier and evaluates them

In [30]:
Y_train.describe()

count    188259.000000
mean      34414.810458
std       23303.179995
min           0.000000
25%        6431.500000
50%       41143.000000
75%       52154.000000
max       86398.000000
Name: remaining_time_FECHA, dtype: float64

In [31]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error

def train_and_evaluate(X_train, X_val, X_test, Y_train, Y_val, Y_test, cols_to_remove = []):
    
    
    # Remove some attributes for training
    train_onehot = X_train.drop(cols_to_remove, axis=1)
    val_onehot = X_val.drop(cols_to_remove, axis=1)
    test_onehot = X_test.drop(cols_to_remove, axis=1)

    # Now, we are going to build estimators for the remaining time
    regressor = RandomForestRegressor(random_state=0,n_jobs=-1)
    regressor.fit(train_onehot, Y_train)

    # Predict
    val_predict = regressor.predict(val_onehot)
    test_predict = regressor.predict(test_onehot)
    
    # And evaluate the model
    val_mae = mean_absolute_error(Y_val, val_predict)
    val_mse = mean_squared_error(Y_val, val_predict)
    test_mae = mean_absolute_error(Y_test, test_predict)
    test_mse = mean_squared_error(Y_test, test_predict)

    print("val: ", val_mae, val_mse )
    print("test: ", test_mae, test_mse)
    
    return val_mae, val_mse, test_mae, test_mse

In [32]:
#columnas a borrar
ctr = ['ID']
print("Start of the experimentation")
train_and_evaluate(X_train, X_val, X_test, Y_train, Y_val, Y_test, cols_to_remove = ctr)

Start of the experimentation
val:  0.43991384038322046 1.1790827627691183
test:  0.4438943806803437 1.2455960989849109


(0.43991384038322046,
 1.1790827627691183,
 0.4438943806803437,
 1.2455960989849109)

In [33]:
#All (ALL): all attributes of the original datasets are considered.
#None (NONE): just the basic attributes of the log, such as name of activity and timestamp are considered.
#Automatic (AUTO): attributes detected as relevant for a decision tree algorithm are selected for the experimentation.
#Random (RND): a set of randomly selected attributes are taken into account.
#Context (CTXT): detected context attributes by the method are included.
#Without context (WCTXT): detected context attributes are excluded from the dataset.

# End classifier